# GAN with lightning

Based on https://lightning.ai/docs/pytorch/stable/notebooks/lightning_examples/basic-gan.html#C.-GAN

---

## Imports

In [1]:
import os

import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from monai.transforms import EnsureChannelFirst, LoadImage
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST

PATH_DATASETS = './datasets'
BATCH_SIZE = 256 if torch.cuda.is_available() else 64
NUM_WORKERS = int(os.cpu_count() / 2)

## Module declaration

In [2]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(
        self,
        data_dir: str = PATH_DATASETS,
        batch_size: int = BATCH_SIZE,
        num_workers: int = NUM_WORKERS,
    ):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
                # LoadImage(),
                # EnsureChannelFirst(),
            ]
        )

        self.dims = (1, 28, 28)
        self.num_classes = 10

    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(
            self.mnist_train,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers)

## Generator

In [3]:
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super().__init__()
        self.img_shape = img_shape

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.01, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh(),
        )

    def forward(self, z):
        print(z.shape)
        print(z.size())
        print(z.dtype)
        print(z.numel())
        img = self.model(z)
        img = img.view(img.size(0), *self.img_shape)
        return img

## Discriminator

In [4]:
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

## GAN implementation

In [5]:
class GAN(pl.LightningModule):
    def __init__(
        self,
        channels,
        width,
        height,
        latent_dim: int = 100,
        lr: float = 0.0002,
        b1: float = 0.5,
        b2: float = 0.999,
        batch_size: int = BATCH_SIZE,
        **kwargs,
    ):
        super().__init__()
        print('GAN.init')
        self.save_hyperparameters()
        self.automatic_optimization = False

        # networks
        data_shape = (channels, width, height)
        print(data_shape)
        self.generator = Generator(latent_dim=self.hparams.latent_dim, img_shape=data_shape)
        self.discriminator = Discriminator(img_shape=data_shape)

        self.validation_z = torch.randn(8, self.hparams.latent_dim)

        self.example_input_array = torch.zeros(2, self.hparams.latent_dim)

    def forward(self, z):
        print('GAN.forward')
        return self.generator(z)

    def adversarial_loss(self, y_hat, y):
        return F.binary_cross_entropy(y_hat, y)

    def training_step(self, batch):
        print('training step...')
        imgs, _ = batch

        optimizer_g, optimizer_d = self.optimizers()

        # sample noise
        z = torch.randn(imgs.shape[0], self.hparams.latent_dim)
        z = z.type_as(imgs)

        # train generator
        # generate images
        self.toggle_optimizer(optimizer_g)
        self.generated_imgs = self(z)

        # log sampled images
        sample_imgs = self.generated_imgs[:6]
        grid = torchvision.utils.make_grid(sample_imgs)
        self.logger.experiment.add_image("train/generated_images", grid, self.current_epoch)

        # ground truth result (ie: all fake)
        # put on GPU because we created this tensor inside training_loop
        valid = torch.ones(imgs.size(0), 1)
        valid = valid.type_as(imgs)

        # adversarial loss is binary cross-entropy
        g_loss = self.adversarial_loss(self.discriminator(self.generated_imgs), valid)
        self.log("g_loss", g_loss, prog_bar=True)
        self.manual_backward(g_loss)
        optimizer_g.step()
        optimizer_g.zero_grad()
        self.untoggle_optimizer(optimizer_g)

        # train discriminator
        # Measure discriminator's ability to classify real from generated samples
        self.toggle_optimizer(optimizer_d)

        # how well can it label as real?
        valid = torch.ones(imgs.size(0), 1)
        valid = valid.type_as(imgs)

        real_loss = self.adversarial_loss(self.discriminator(imgs), valid)

        # how well can it label as fake?
        fake = torch.zeros(imgs.size(0), 1)
        fake = fake.type_as(imgs)

        fake_loss = self.adversarial_loss(self.discriminator(self.generated_imgs.detach()), fake)

        # discriminator loss is the average of these
        d_loss = (real_loss + fake_loss) / 2
        self.log("d_loss", d_loss, prog_bar=True)
        self.manual_backward(d_loss)
        optimizer_d.step()
        optimizer_d.zero_grad()
        self.untoggle_optimizer(optimizer_d)

    def validation_step(self, batch, batch_idx):
        print('validation skipped')
        pass

    def configure_optimizers(self):
        lr = self.hparams.lr
        b1 = self.hparams.b1
        b2 = self.hparams.b2

        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(b1, b2))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(b1, b2))
        return [opt_g, opt_d], []

    def on_validation_epoch_end(self):
        z = self.validation_z.type_as(self.generator.model[0].weight)

        # log sampled images
        sample_imgs = self(z)
        grid = torchvision.utils.make_grid(sample_imgs)
        self.logger.experiment.add_image("validation/generated_images", grid, self.current_epoch)

## Actual training

In [6]:
from monai.utils import first

dm = MNISTDataModule()
model = GAN(*dm.dims)
trainer = pl.Trainer(
    accelerator="auto",
    devices=1,
    max_epochs=1,
)
print(dm)
# print(model)
# summary(model)

dm.prepare_data()
dm.setup()

aux = first(dm.train_dataloader())
print(len(aux))
print(aux[0].size())
print(aux[1].size())
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


GAN.init
(1, 28, 28)
2
torch.Size([64, 1, 28, 28])
torch.Size([64])
GAN.forward
torch.Size([2, 100])
torch.Size([2, 100])
torch.float32
200



  | Name          | Type          | Params | Mode  | In sizes | Out sizes     
------------------------------------------------------------------------------------
0 | generator     | Generator     | 1.5 M  | train | [2, 100] | [2, 1, 28, 28]
1 | discriminator | Discriminator | 533 K  | train | ?        | ?             
------------------------------------------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.174     Total estimated model params size (MB)
23        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

validation skipped
validation skipped
GAN.forward
torch.Size([8, 100])
torch.Size([8, 100])
torch.float32
800


Training: |          | 0/? [00:00<?, ?it/s]

training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.float32
6400
training step...
GAN.forward
torch.Size([64, 100])
torch.Size([64, 100])
torch.f

Validation: |          | 0/? [00:00<?, ?it/s]

validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation skipped
validation s

`Trainer.fit` stopped: `max_epochs=1` reached.
